## 學生類各表（36張）

In [2]:
import pandas as pd
from pathlib import Path
import re
import os

In [3]:
projRoot = Path.cwd().parents[0]
root_rawdata = projRoot.joinpath('data/raw')

In [4]:
filepaths = sorted(root_rawdata.rglob('學*.csv'))
print(filepaths)

[PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學1-1.正式學籍在學學生人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學1-2.正式學籍在學學生人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學10.日間學制修讀雙聯學制學生數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學11.學校辦理國際合作與交流資料表-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-1.新生(含境外生)註冊率-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-2.新生(含境外生)註冊率-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-3.新生(含境外生)註冊率-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學13-1.於學年底處於休學狀態之人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學13-2.於學年底處於休學狀態之人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學14-1.退學人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學14-2.退學人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac

### Stage 1 & Stage 2 ( '...' to np.nan, object to int, & aggregate to school level )

In [5]:
# Stage 1
import numpy as np

root_stage1 = projRoot.joinpath('data/stage1')
root_stage1.mkdir(exist_ok=True)

for fn in filepaths:
    csv = pd.read_csv(fn, na_values=['...'])
    int_columns = csv.select_dtypes(include=['int', 'float']).columns.tolist()
    int_columns = list(filter(lambda x: x not in ['學年度', '學校統計處代碼'], int_columns))
    if '學期' in list(csv.columns):
        csv = csv.groupby(['學年度', '學校統計處代碼', '學期'])[int_columns].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
        csv.reset_index(inplace=True)
        csv = csv.pivot(index=['學年度', '學校統計處代碼'], columns= '學期', values= int_columns)
        csv.columns = [f"{col[0]}_{col[1]}" for col in csv.columns]
        csv.reset_index(inplace=True)
        csv = csv.drop(['學期_1', '學期_2'], axis=1)
    elif '學年底' in f'{fn}':
        csv = csv.rename(columns={col: col+'_2' for col in csv[int_columns]})

    csv_filename = f'{root_stage1}/{fn.name}_stage1.csv'
    csv.to_csv(csv_filename, index=False)

/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_52819/346681178.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv(fn, na_values=['...'])


In [8]:
root_stage1data = projRoot.joinpath('data/stage1')
filepaths_1 = sorted(root_stage1data.rglob('學*_stage1.csv'))
print(filepaths)

[PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學1-1.正式學籍在學學生人數-以「系(所)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學1-2.正式學籍在學學生人數-以「校(含學制班別)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學10.日間學制修讀雙聯學制學生數-以「系(所)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學11.學校辦理國際合作與交流資料表-以「校」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學12-1.新生(含境外生)註冊率-以「系(所)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學12-2.新生(含境外生)註冊率-以「校(含學制班別)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學12-3.新生(含境外生)註冊率-以「校」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學13-1.於學年底處於休學狀態之人數-以「系(所)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學13-2.於學年底處於休學狀態之人數-以「校(含學制班別)」統計.csv_stage1.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/學14-1.退學

In [9]:
# Stage 2

root_stage2 = projRoot.joinpath('data/stage2')
root_stage2.mkdir(exist_ok=True)

for fn in filepaths_1:
    csv = pd.read_csv(fn)
    # columns to aggregate (those that are int or float)
    int_columns = csv.select_dtypes(include=['int', 'float']).columns.tolist()
    int_columns = list(filter(lambda x: x not in ['學年度', '學校統計處代碼'], int_columns))
    # aggregate
    csv = csv.groupby(['學年度', '學校統計處代碼'])[int_columns].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
    cols_to_drop = [col for col in csv.columns if '%' in col]
    csv = csv.drop(cols_to_drop, axis=1)
    csv.reset_index(inplace=True)
    csv_filename = f'{root_stage2}/{fn.name}_stage2.csv'
    csv.to_csv(csv_filename, index=False)
    print(cols_to_drop)

[]
[]
[]
[]


/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_52819/1375668822.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv(fn)


['當學年度新生註冊率(%)D=〔(C+E)/(A-B+E)〕＊100％']
['當學年度各學制新生註冊率(%)E=〔(C+D)/(A-B+D)〕*100％']
[]
[]
[]
[]
[]
[]
['前一學年度學生就學穩定率(%)(C)(C=B/A)']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['境外學位生數之在學比率(%)']
['外國學生數之在學比率(%)']
['僑生及港澳生數之在學比率(%)']
['大陸地區來臺學位生數之在學比率(%)']
[]
[]
[]
[]
[]
[]
[]
[]
[]


### Stage 3 ( long form data, all merged together )

In [5]:
# Stage 3
dfs = {}
for file in files:
    filename = file.split('/')[-1]
    dfs[filename] = pd.read_csv(file)
    dfs[filename][['學年度', '學校統計處代碼']] = dfs[filename][['學年度', '學校統計處代碼']].astype(object)

# merge all 
keys = list(dfs.keys())
merged_df = dfs[keys[0]]
for i in range(1, len(keys)):
    merged_df = pd.merge(merged_df, dfs[keys[i]], how='outer')